In [1]:
import requests
#from urllib.robotparser import RobotFileParser
from bs4 import BeautifulSoup
import urllib3
import warnings
warnings.filterwarnings('ignore')
import datetime as dtime
import pandas as pd
import re
import csv
import sqlite3 as sq3

#### Simple request

In [2]:
#ingatlan.hu, one page
def simpleget(url):
    r=requests.get(url, headers={'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:62.0) Gecko/20100101 Firefox/62.0'})
    
    page = BeautifulSoup(r.content, 'lxml')
    body = page.find('body')
    return body

#load from csv to pandas



#### Multi-page search retrieval

In [3]:
#ingatlan.com, multi-page
def inner_multi_get(url, n):
    link = url + str(n)
    print(link)
    http = urllib3.PoolManager()
    response = http.request('GET', link)
    soup = BeautifulSoup(response.data, 'lxml')
    body = soup.find_all('main')
    return body

def multipages(url , v=1):
    i=1
    while i < v+1:
        text = inner_multi_get(url, n=i)
        
        if 'csvtext' in locals():
            csvtext = csvtext + text
        else: csvtext = text
        i += 1
    return csvtext

#### Saving html source data into file

In [4]:
def writeto(csvtext, fnev): 
    file = open(fnev, 'w')
    file.write('%s' % csvtext)
    file.close()   
    return print('Data saved as %s' % fnev )

def savetext(csvtext,fnev, n=0):
    # file name includes day-month timestamp and page number for multi-page retievals
    now = dtime.datetime.now()
    m = str(now.month)
    d = str(now.day)
    y = str(now.year)
    fn = fnev + '_' + d + m + y + '.csv'
    writeto(csvtext,fn)   
    return 

#### Getting data from html

In [5]:
def mining_com(csvtext):
    data = {'id':[],'price':[],'sqm':[],'address':[],'area':[],'room':[],'date':[],'listing_count':[]}
    df = pd.DataFrame()
    for i in csvtext:
        a = i.find_all('div', attrs={'class': re.compile('listing js-listing*')})
        print ('Ads on the page: %d' % len(a))
        data['id']= data['id']+[ap.attrs['data-id'] for ap in i.find_all('div', attrs={'class':re.compile('listing js-listing*')})]
        data['price']= data['price']+[p.text for p in i.find_all('div', attrs={'class':'price'})]
        data['sqm']=data['sqm']+[s.text for s in i.find_all('div', attrs={'class':'price--sqm'})]
        data['address']=data['address']+[a.text for a in i.find_all('div', attrs={'class':'listing__address'})]
        data['area']=data['area']+[a.text for a in i.find_all('div', attrs={'class':'listing__parameter listing__data--area-size'})]
        data['room']=data['room']+[a.text for a in i.find_all('div', attrs={'class':'listing__parameter listing__data--room-count'})]
        data['date']=data['date']+[dtime.datetime.now().strftime("%Y-%m-%d") for a in i.find_all('div', attrs={'class':'price'})]
        data['listing_count']=data['listing_count']+[1 for a in i.find_all('div', attrs={'class':'price'})]
    for k in data.keys():
        df[k]=data[k][:len(data['id'])]
    
    return df


def mining_hu(csvtext):
    
    return

#### Append to database

In [6]:
def tosql(sql,df):
    for i in df.index:
        row = df.iloc[i]
        sql = sql.append(row, ignore_index=True)

        for ii, v in enumerate(sql.duplicated(subset=['address','area','room'], keep=False)):

            if v == True:
                sql['listing_count'].iloc[ii] = sql['listing_count'].iloc[ii] + 1
                sql=sql[:-1]
                break
    return sql

#### Saving to sql database

In [7]:
def connect(dbname, dbview='no'):
    conn = sq3.connect(dbname)
    if dbview == 'yes':
        print(conn.execute('select * FROM sqlite_master').fetchall())
    return conn

def query(q, dbname, df='yes'):
    if df == 'no':
        conn=connect(dbname)
        conn.execute(q)
        print(conn.execute('select * FROM sqlite_master').fetchall())
        conn.close()
    else:
        conn = connect(dbname)
        q = pd.read_sql(q, conn)
        conn.close()
        return q
    
def dfinsert(q, dbname, table):
    conn = connect(dbname)
    q.to_sql(table,conn, if_exists='append', index=False)
    rq='SELECT * FROM'+table
    r = query(rq, conn)
    n= len(r)
    conn.close()
    return n

In [8]:
query = '''
CREATE TABLE ingatlanCom (
    id,price,sqm,address,area,room,date,listing_count
    )
'''

### Set-up

In [9]:
def retriveall_tofile():
    url1 = 'https://ingatlan.com/szukites/elado+lakas+v-ker+25-33-m2+ar-szerint?page='
    url1a = 'https://ingatlan.com/szukites/elado+lakas+xi-ker+48-55-m2+ar-szerint?page='
    url1b = 'https://ingatlan.com/szukites/elado+lakas+vi-ker+38-55-m2+ar-szerint?page='
    url2 = 'https://ingatlanok.hu/elado/lakas+haz/budapest-v-ker:25nm-tol;33nm-ig?record=0&num=100'
    url2a = 'https://ingatlanok.hu/elado/lakas+haz/budapest-xi-ker:48nm-tol;55nm-ig?record=0&num=100'
    url2b = 'https://ingatlanok.hu/elado/lakas+haz/budapest-vi-ker:38nm-tol;55nm-ig?record=0&num=100'
    url3 = 'https://www.oc.hu/ingatlanok/lista/oldalszam:48/rendezes:relevance/felhasznalas:elado/jogi-status:hasznalt/tipus:flat/stilus:brick,/hely-ertek:budapest05-kerulet/hely-id:budapest05-kerulet,/netto-alapterulet:28~33/emelet:1,8'
    url3a = 'https://www.oc.hu/ingatlanok/lista/oldalszam:48/rendezes:relevance/felhasznalas:elado/jogi-status:hasznalt/tipus:flat/stilus:brick,/hely-ertek:budapest11-kerulet/hely-id:budapest11-kerulet,/netto-alapterulet:45~55/emelet:1,8'
    url3b = 'https://www.oc.hu/ingatlanok/lista/oldalszam:48/rendezes:relevance/felhasznalas:elado/jogi-status:hasznalt/tipus:flat/stilus:brick,/hely-ertek:budapest06-kerulet/hely-id:budapest06-kerulet,/netto-alapterulet:38~55/emelet:1,8'
    
    hu = {
        'ingatlanokHU_Vker': url2, 
        'ingatlanokHU_XIker': url2a,
        'ingatlanokHU_VIker': url2b,
        'otthoncentrum_Vker': url3,
        'otthoncentrum_XIker': url3a,
        'otthoncentrum_VIker': url3b
    }
    com = {
        'ingatlanCOM_Vker': url1,
        'ingatlanCOM_XIker': url1a,
        'ingatlanCOM_VIker': url1b,

    }
    
    #one page download
    for i in hu.keys():
        csvtext = simpleget(hu[i])
        savetext(csvtext, i)

    #multipage download
    for i in com.keys():
        csvtext = multipages(com[i], v=15)
        savetext(csvtext, i)
    

#### Runing the show

In [10]:
retriveall_tofile()

Data saved as ingatlanokHU_Vker_3092019.csv
Data saved as ingatlanokHU_XIker_3092019.csv
Data saved as ingatlanokHU_VIker_3092019.csv
Data saved as otthoncentrum_Vker_3092019.csv
Data saved as otthoncentrum_XIker_3092019.csv
Data saved as otthoncentrum_VIker_3092019.csv
https://ingatlan.com/szukites/elado+lakas+v-ker+25-33-m2+ar-szerint?page=1
https://ingatlan.com/szukites/elado+lakas+v-ker+25-33-m2+ar-szerint?page=2
https://ingatlan.com/szukites/elado+lakas+v-ker+25-33-m2+ar-szerint?page=3
https://ingatlan.com/szukites/elado+lakas+v-ker+25-33-m2+ar-szerint?page=4
https://ingatlan.com/szukites/elado+lakas+v-ker+25-33-m2+ar-szerint?page=5
https://ingatlan.com/szukites/elado+lakas+v-ker+25-33-m2+ar-szerint?page=6
https://ingatlan.com/szukites/elado+lakas+v-ker+25-33-m2+ar-szerint?page=7
https://ingatlan.com/szukites/elado+lakas+v-ker+25-33-m2+ar-szerint?page=8
https://ingatlan.com/szukites/elado+lakas+v-ker+25-33-m2+ar-szerint?page=9
https://ingatlan.com/szukites/elado+lakas+v-ker+25-33-

In [213]:
dbname='houseprice.db'
#query(q2, dbname, 'yes')
connect(dbname, 'yes')

[]


In [211]:
q1 = """ select count(*) from ingatlanCom"""
q2 = """select distinct(date) from ingatlanCom"""
q3 = "drop table ingatlanCom"

In [212]:
query(q3,dbname)

TypeError: 'NoneType' object is not iterable

In [184]:
#elso file a Com oldalrol
fname = 'ingatlanCom10.html'
f=open(fname, 'r').read()

soup = BeautifulSoup(f)
csvtext = soup.find_all('div', attrs={'class': re.compile('listing js-listing*')})

data = {}
data['id'] = [i['data-id'] for i in csvtext]
data['price M Ft']=[i.find('sup').next.next.next.split(' ')[1] for i in csvtext]
data['Ft/m2']=[i.find(attrs={'class':'listing__price--sqm'}).text.split(' ')[1:-2] for i in csvtext]
data['address'] =[i.find('div', attrs={'class':'listing__address'}).text for i in csvtext]
data['district'] =[i.find('div', attrs={'class':'listing__address'}).text.split(' ')[-2] for i in csvtext]
data['area m2'] = [i.find('span',attrs={'class':'listing__data-number'}).text for i in csvtext]
data['room'] = [i.find(attrs={'class':'listing__save js-press js-save-listing'}).previous\
.previous.previous.previous for i in csvtext]
data['date'] = [dtime.datetime(2018,9,10).strftime("%Y-%m-%d") for i in csvtext]
data['listing_count']= [1 for i in csvtext]

cols = data.keys()
dfCom  = pd.DataFrame(data, columns= cols).sort_values('area').reset_index(drop=True)
dfCom


,id,price M Ft,Ft/m2,address,district,area,room,date,listing_count
0,26525464,24,"[960, 000]","M\xc3\xa9nesi \xc3\xbat, XI. ker\xc3\xbclet",XI.,25,1,2018-09-10,1
1,27611101,24.9,"[996, 000]","Kondorosi \xc3\xbat, XI. ker\xc3\xbclet",XI.,25,1,2018-09-10,1
2,27584202,21.99,"[879, 600]","Feh\xc3\xa9rv\xc3\xa1ri \xc3\xbat 168-178/B, ...",XI.,25,1 + 1 f\xc3\xa9l,2018-09-10,1
3,27581139,23.4,"[866, 667]","Kelenf\xc3\xb6ldi \xc3\xbat 30, XI. ker\xc3\x...",XI.,27,1,2018-09-10,1
4,27363186,23.5,"[870, 370]","K\xc3\xb6b\xc3\xb6lk\xc3\xbat utca, XI. ker\x...",XI.,27,1,2018-09-10,1
5,27596757,21,"[750, 000]","Nagyszeben t\xc3\xa9r, XI. ker\xc3\xbclet",XI.,28,1,2018-09-10,1
6,27383247,34.15,"[1, 219, 660]","Havas utca, V. ker\xc3\xbclet",V.,28,1,2018-09-10,1
7,27395082,33.99,"[1, 172, 069]","Balassi B\xc3\xa1lint utca, V. ker\xc3\xbclet",V.,29,1,2018-09-10,1
8,27650968,22.9,"[750, 820]","Gazdagr\xc3\xa9ti t\xc3\xa9r, XI. ker\xc3\xbclet",XI.,29,1,2018-09-10,1
9,26941093,26.9,"[867, 742]","K\xc3\xb6k\xc3\xb6rcsin utca, XI. ker\xc3\xbclet",XI.,30,1,2018-09-10,1


In [186]:
# elso file a HU oldalrol
f = open('ingatlanHU0910.html', 'r').read()

soup = BeautifulSoup(f)
csvtext = soup.find_all(attrs={'class':'result-row-container'})

data ={}
data['district'] =[ i.find(attrs={'class':'result-title left'}).text.split(' ')[3] for i in csvtext]
data['price '] = [i.find(attrs={'class':'result-price left'}).find('a').text.split(' ')[0] for i in csvtext]
data['area m2'] = [i.find(attrs={'class':'column-first'}).find('a').next.next.next.text.split(' ')[0] for i in csvtext]
data['rooms'] = [i.find(attrs={'class':'column'}).find('a').next.next.next.next for i in csvtext]
data['type'] = [i.find(attrs={'class':'result-item-bottom'}).previous.previous.previous.previous.previous.previous.previous for i in csvtext]
data['decription'] = [i.find(attrs={'class':'result-short-description'}).text for i in csvtext]
data['date'] = [dtime.datetime(2018,9,10).strftime("%Y-%m-%d") for i in csvtext]

cols = data.keys()
dfHU= pd.DataFrame(data, columns=cols)
dfHU

In [210]:
for k,v in enumerate(data):
    print('%s: %s' % (str(v), str(len(data[v]))))

district: 41
price : 41
area m2: 41
rooms: 41
type: 41
decription: 41
date: 41


,id,price,sqm,address,area,room,date,listing_count
0,27755537,11.5 M Ft,370 968 Ft/m2,"Hadak útja, XI. kerület",31 m² terület,1 szoba,2018-10-05,1
1,27666949,17.5 M Ft,530 303 Ft/m2,"Könyves György utca, XI. kerület",33 m² terület,1 szoba,2018-10-05,1
2,25717112,17.5 M Ft,673 077 Ft/m2,"Bánk bán utca, XI. kerület",26 m² terület,1 szoba,2018-10-05,1
3,27848696,19.49 M Ft,628 710 Ft/m2,"Hengermalom út, XI. kerület",31 m² terület,1 szoba,2018-10-05,1
4,27814889,19.49 M Ft,628 710 Ft/m2,"Hengermalom út, XI. kerület",31 m² terület,1 szoba,2018-10-05,1
5,27797285,19.49 M Ft,628 710 Ft/m2,"Hengermalom út, XI. kerület",31 m² terület,1 szoba,2018-10-05,1
6,27729743,19.49 M Ft,628 710 Ft/m2,"Hengermalom út, XI. kerület",31 m² terület,1 szoba,2018-10-05,1
7,27649228,19.49 M Ft,628 710 Ft/m2,"Hengermalom út, XI. kerület",31 m² terület,1 szoba,2018-10-05,1
8,27733610,19.5 M Ft,577 778 Ft/m2,"Albertfalvai tisztviselőtelep, XI. kerület",33 m² terület,1 szoba,2018-10-05,1
9,27276313,19.9 M Ft,663 333 Ft/m2,"Ballagi Mór utca, XI. kerület",30 m² terület,1 szoba,2018-10-05,1
